In [2]:
print( "this is assignment 6 by Mohammad Movahedi")

this is assignment 6 by Mohammad Movahedi


In [44]:
# I'm going to analyze the book "Meditations" by Marcus Aurelius

# Reading a local text file
with open('Meditations.txt', 'r', encoding='utf-8') as file:
    meditations_text = file.read() # read the file
    

In [51]:
import nltk 
from nltk.tokenize import word_tokenize, sent_tokenize , RegexpTokenizer
nltk.download('punkt')

# Tokenize words
words = word_tokenize(meditations_text)

# Get number of words
num_words = len(words)

# Tokenize sentences
sentences = sent_tokenize(meditations_text)

# For a single file, you can assign a custom ID
file_id = 'meditations'

# You have only one file
num_files = 1

# Raw text sample
raw_text_sample = meditations_text.strip()[:1000]

print(f"Number of words: {num_words}")
print(f"Number of sentences: {len(sentences)}")
print(f"Number of files: {num_files}")
print(f"Custom File ID: {file_id}")
#print(f"Raw text sample: {raw_text_sample}")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of words: 87711
Number of sentences: 3438
Number of files: 1
Custom File ID: meditations
Raw text sample: The Project Gutenberg eBook of Meditations
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Meditations


Author: Emperor of Rome Marcus Aurelius

Release date: June 1, 2001 [eBook #2680]
                Most recently updated: March 9, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK MEDITATIONS ***



MEDITATIONS

By Marcus Aurelius




CONTENTS


     NOTES

     INTRODUCTION

     FIRST BOOK

     SECOND BOOK

     THIRD BOOK

     FOURTH BOOK

     FIFT

In [49]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from string import punctuation

# Function for POS tag conversion
def penn2morphy(penntag):
    return {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'}.get(penntag[:2], 'n')

# Function for text lemmatization
def lemmatize_sent(text):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(word_tokenize(text))]

# Combined stopwords
stoplist_combined = set(stopwords.words('english')).union(punctuation)

# Preprocess and lemmatize the 100th line
line_no100 = meditations_text.split('\n')[99]
lemmatized_line_no100 = [word for word in lemmatize_sent(line_no100) if word not in stoplist_combined]

# Count word occurrences and create term frequency matrix
word_counts = Counter(lemmatized_line_no100)
count_vect = CountVectorizer(analyzer=lemmatize_sent)
term_freq_matrix = count_vect.fit_transform([line_no100])

print("Lemmatized line_no100:", lemmatized_line_no100)
print("Word counts:", word_counts)
print("Term Frequency Matrix:", term_freq_matrix.toarray())


Lemmatized line_no100: ['extravagancy', 'day', 'great', 'excitement', 'rome', 'strife']
Word counts: Counter({'extravagancy': 1, 'day': 1, 'great': 1, 'excitement': 1, 'rome': 1, 'strife': 1})
Term Frequency Matrix: [[1 1 1 1 1 1 1 1 1 1 1 2]]


<h1>Objective</h1>
I'm aiming to build a Naive Bayes classifier that can categorize sections of the "Meditations" text into different philosophical themes, such as 'ethics,' 'self-control,' or 'duty.'

In [79]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Define the text cleaning function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    return text

# Initialize the WordNet Lemmatizer
wnl = WordNetLemmatizer()

# Initialize stop words
stop_words = set(stopwords.words('english'))

# Read and clean the text
sections = meditations_text.split("BOOK")[1:]
cleaned_sections = [clean_text(section) for section in sections]

# Tokenize, remove stop words and lemmatize
tokenized_and_cleaned_sections = []
for section in cleaned_sections:
    tokens = word_tokenize(section)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatized_tokens = [wnl.lemmatize(word) for word in filtered_tokens]
    tokenized_and_cleaned_sections.append(lemmatized_tokens)



In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Convert the list of words for each section back to string format
sections_as_str = [' '.join(section) for section in tokenized_and_cleaned_sections]

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Limit the max_features in TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the cleaned sections
X = vectorizer.fit_transform(sections_as_str)

# Refit LDA model with new data
best_lda_model.fit(X)

# Get the new topic distribution
topic_distribution = best_lda_model.transform(X)
labels = [np.argmax(dist) for dist in topic_distribution]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Initialize Naive Bayes classifier
clf = MultinomialNB()

# Train the classifier
clf.fit(X_train, y_train)

# Test the classifier
y_pred = clf.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7142857142857143
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.71      1.00      0.83         5
           2       0.00      0.00      0.00         1

    accuracy                           0.71         7
   macro avg       0.24      0.33      0.28         7
weighted avg       0.51      0.71      0.60         7



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SVM classifier
clf = SVC()

# Train the classifier
clf.fit(X_train, y_train)

# Predict labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))


Accuracy:  0.7142857142857143
Classification Report: 
               precision    recall  f1-score   support

           1       0.71      1.00      0.83         5
           2       0.00      0.00      0.00         2

    accuracy                           0.71         7
   macro avg       0.36      0.50      0.42         7
weighted avg       0.51      0.71      0.60         7



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



The SVM model achieved a 71.43% accuracy on a small test set of 7 samples.